# Inverse designs

In this notebook we will investigate a complex design created by topology optimization. The specific design is the `Rasmus70nm.csv` file from the RGB metalens problem of the [photonics-opt-testbed repo](https://github.com/NanoComp/photonics-opt-testbed).

In [ ]:
import numpy as onp
import matplotlib.pyplot as plt
from skimage import measure

design = onp.genfromtxt("../../reference_designs/Rasmus70nm.csv", delimiter=",")
design = design > 0.5  # Binarize
design = onp.rot90(design)

plt.figure(figsize=(12, 4))
ax = plt.subplot(111)
im = ax.imshow(1 - design, cmap="gray")
im.set_clim([-2, 1])
for c in measure.find_contours(design):
    ax.plot(c[:, 1], c[:, 0], "k")
_ = ax.axis(False)

Now we'll measure the length scale of solid and void features (i.e. the minimum width and spacing of features) using the imageruler.

In [ ]:
import imageruler

min_width, min_spacing = imageruler.minimum_length_scale(design)
print(f"Minimum width is {min_width} pixels, minimum spacing is {min_spacing} pixels.")

We find that the minimum width is 7 pixels, which means that we should find violations when checking for length scales greater than 7 pixels. We can identify these violations using the `length_scale_violations_solid` function:

In [ ]:
from scipy import ndimage

plt.figure(figsize=(12, 6))
for i, dim in enumerate(range(min_width, min_width + 3)):
    width_violations = imageruler.length_scale_violations_solid(
        design, length_scale=dim
    )
    width_violations = ndimage.binary_dilation(width_violations, onp.ones((10, 10)))

    ax = plt.subplot(3, 1, i + 1)
    im = ax.imshow(1 - design, cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(design):
        ax.plot(c[:, 1], c[:, 0], "k")

    for c in measure.find_contours(width_violations):
        ax.fill(c[:, 1], c[:, 0], "r", alpha=0.5, ec="r")

    ax.set_title(f"Violations for solid length scale = {dim}")
    ax.axis(False)

And similarly, we can find void length scale violations:

In [ ]:
plt.figure(figsize=(12, 6))
for i, dim in enumerate(range(min_spacing, min_spacing + 3)):
    width_violations = imageruler.length_scale_violations_solid(
        ~design, length_scale=dim
    )
    width_violations = ndimage.binary_dilation(width_violations, onp.ones((10, 10)))

    ax = plt.subplot(3, 1, i + 1)
    im = ax.imshow(1 - design, cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(design):
        ax.plot(c[:, 1], c[:, 0], "k")

    for c in measure.find_contours(width_violations):
        ax.fill(c[:, 1], c[:, 0], "r", alpha=0.5, ec="r")

    ax.set_title(f"Violations for void length scale = {dim}")
    ax.axis(False)